In [1]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
handler.addFilter(logging.Filter("trulens"))
handler.setFormatter(formatter)
root.addHandler(handler)

In [2]:
# Create snowpark session.

from snowflake.snowpark import Session
from trulens.connectors.snowflake import SnowflakeConnector

snowflake_connection_parameters = {
    "account": "SNOWHOUSE",
    "user": "dhuang",
    "database": "SNOWFLAKE_INTELLIGENCE",
    "schema": "AGENTS",
    "authenticator": "externalbrowser",
}
snowpark_session = Session.builder.configs(
    snowflake_connection_parameters
).create()

sf_connector = SnowflakeConnector(snowpark_session=snowpark_session)

/opt/anaconda3/envs/trulens/lib/python3.11/site-packages/snowflake/snowpark/session.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


2025-10-04 01:03:19,010 - trulens.connectors.snowflake.connector - WARNING - Running the TruLens dashboard requires providing a `password` to the `SnowflakeConnector`.
2025-10-04 01:03:19,010 - trulens.connectors.snowflake.dao.external_agent - INFO - Initialized ExternalAgentDao with a Snowpark session.


In [3]:
df = snowpark_session.sql("""
    SELECT
        *
    FROM TABLE(SNOWFLAKE.LOCAL.GET_AI_OBSERVABILITY_EVENTS(
        CURRENT_DATABASE(), 
        CURRENT_SCHEMA(), 
        'SNOWFLAKE_DOCS_AND_KNOWLEDGE', 
        'CORTEX AGENT'
    ))
""").to_pandas()


In [42]:


df.iloc[-1]['RECORD_ATTRIBUTES']

'{\n  "ai.observability.record_id": "57ec98b7-57de-4b05-abff-1560da5003c4",\n  "snow.ai.observability.agent.duration": 10857,\n  "snow.ai.observability.agent.first_message_in_thread": "can i create a clone of a table and add lcustering keys?",\n  "snow.ai.observability.agent.messages": "ALTER TABLE PLAIDTRANSACTION_PARSED_COPY_SOpt \\nADD SEARCH OPTIMIZATION ON (PLAIDACCOUNTID);\\n\\nthis failed",\n  "snow.ai.observability.agent.parent_message_id": 2048701229170,\n  "snow.ai.observability.agent.request_id": "57ec98b7-57de-4b05-abff-1560da5003c4",\n  "snow.ai.observability.agent.research_mode": false,\n  "snow.ai.observability.agent.response": "\\nI need to see the specific error message to help troubleshoot. Can you share the exact error you received?\\n\\n**Common issues and solutions:**\\n\\n**1. Check if table exists:**\\n```sql\\nSHOW TABLES LIKE \'PLAIDTRANSACTION_PARSED_COPY_SOpt\';\\n```\\n\\n**2. Check column name (case sensitive):**\\n```sql\\nDESCRIBE TABLE PLAIDTRANSACTION_P

In [110]:
def get_spans_by_filters(snowpark_session, 
                        app_name='SNOWFLAKE_DOCS_AND_KNOWLEDGE',
                        thread_id=None, 
                        request_id=None):
    
    base_query = """
        SELECT
            *
        FROM TABLE(SNOWFLAKE.LOCAL.GET_AI_OBSERVABILITY_EVENTS(
            CURRENT_DATABASE(), 
            CURRENT_SCHEMA(), 
            ?, 
            'CORTEX AGENT'
        ))
        WHERE RECORD_TYPE = 'SPAN'
    """
    
    where_clauses = []
    params = [app_name]
    
    if thread_id:
        where_clauses.append('RECORD_ATTRIBUTES:"snow.ai.observability.agent.thread_id" = ?')
        params.append(str(thread_id))
    
    if request_id:
        where_clauses.append('RECORD_ATTRIBUTES:"ai.observability.record_id" = ?')
        params.append(request_id)
    
    if where_clauses:
        query = base_query + " AND " + " AND ".join(where_clauses)
    else:
        query = base_query
    
    query += " ORDER BY START_TIMESTAMP ASC"
    
    return snowpark_session.sql(query, params=params).to_pandas()

# Usage examples:
# Get all spans for a thread
df_thread = get_spans_by_filters(snowpark_session, thread_id='2048699676202')

# Get spans for a specific request
df_request = get_spans_by_filters(snowpark_session, request_id='52f45c9c-5c91-4e46-8804-ea6896410e6e')

# Get spans for both (most specific)
df_both = get_spans_by_filters(snowpark_session, 
                              thread_id='2048699676202',
                              request_id='52f45c9c-5c91-4e46-8804-ea6896410e6e')

In [116]:
json.loads(df_request['TRACE'].iloc[0])

{'span_id': '61e0f7c2253a7cbe', 'trace_id': '0266ef2904ab2c46c3697d75b056ce5b'}

In [127]:
def analyze_trace(trace_df):
    """Analyze the execution trace with correct parent_span_id location"""
    if trace_df.empty:
        return "No spans found"
    
    print(f"Total spans: {len(trace_df)}")
    print(f"Trace duration: {trace_df['START_TIMESTAMP'].min()} to {trace_df['START_TIMESTAMP'].max()}")
    
    # Parse and show span hierarchy
    for idx, row in trace_df.iterrows():
        trace_info = json.loads(row['TRACE'])
        record_info = json.loads(row['RECORD'])  # parent_span_id is here
        record_attrs = json.loads(row['RECORD_ATTRIBUTES'])
        
        span_id = trace_info.get('span_id', 'unknown')
        parent_span_id = record_info.get('parent_span_id', 'root')  # Correct location
        span_name = record_info.get('name', 'unknown')
        
        print(f"Span {idx+1}: {span_name} ({span_id}) -> parent: {parent_span_id} - {row['START_TIMESTAMP']}")
    
    return trace_df


def get_execution_trace(snowpark_session, 
                       app_name='SNOWFLAKE_DOCS_AND_KNOWLEDGE',
                       thread_id=None, 
                       request_id=None):
    
    base_query = """
        SELECT
            *
        FROM TABLE(SNOWFLAKE.LOCAL.GET_AI_OBSERVABILITY_EVENTS(
            CURRENT_DATABASE(), 
            CURRENT_SCHEMA(), 
            ?, 
            'CORTEX AGENT'
        ))
        WHERE RECORD_TYPE = 'SPAN'
    """
    
    where_clauses = []
    params = [app_name]
    
    if thread_id:
        where_clauses.append('RECORD_ATTRIBUTES:"snow.ai.observability.agent.thread_id" = ?')
        params.append(str(thread_id))
    
    if request_id:
        where_clauses.append('RECORD_ATTRIBUTES:"ai.observability.record_id" = ?')
        params.append(request_id)
    
    if where_clauses:
        query = base_query + " AND " + " AND ".join(where_clauses)
    else:
        query = base_query
    
    # Add ordering for trace assembly
    query += " ORDER BY START_TIMESTAMP ASC"
    
    return snowpark_session.sql(query, params=params).to_pandas()

# Usage examples:
# Get complete execution trace for a request (chronologically ordered)


trace_df = get_execution_trace(snowpark_session, request_id='52f45c9c-5c91-4e46-8804-ea6896410e6e')
analyzed_trace = analyze_trace(trace_df)

Total spans: 8
Trace duration: 2025-10-04 08:56:51.344000 to 2025-10-04 08:57:23.944000
Span 1: Agent (61e0f7c2253a7cbe) -> parent: root - 2025-10-04 08:56:51.344000
Span 2: AgentV2RequestResponseInfo (5abaeed1c0fdbdf3) -> parent: 61e0f7c2253a7cbe - 2025-10-04 08:56:51.344000
Span 3: ReasoningAgentStepPlanning-0 (ff4703779def7ff1) -> parent: 61e0f7c2253a7cbe - 2025-10-04 08:56:51.358000
Span 4: CortexAnalystTool_Workday_Orgchart (dc4c7cb75c4870b6) -> parent: 61e0f7c2253a7cbe - 2025-10-04 08:57:01.314000
Span 5: SqlExecution_CortexAnalyst (2aaa6d8c43aa9091) -> parent: dc4c7cb75c4870b6 - 2025-10-04 08:57:10.545000
Span 6: ReasoningAgentStepPlanning-1 (af94ca0da255e616) -> parent: 61e0f7c2253a7cbe - 2025-10-04 08:57:13.762000
Span 7: CortexChartToolImpl-data_to_chart (a8d9f15994a06d72) -> parent: 61e0f7c2253a7cbe - 2025-10-04 08:57:21.250000
Span 8: ReasoningAgentStepResponseGeneration-2 (6cfac07e656f92ab) -> parent: 61e0f7c2253a7cbe - 2025-10-04 08:57:23.944000


In [ ]:
def build_span_hierarchy_with_content(trace_df):
    """Build a hierarchical view of the spans with concatenated record_attributes content"""
    spans = {}
    
    for idx, row in trace_df.iterrows():
        trace_info = json.loads(row['TRACE'])
        record_info = json.loads(row['RECORD'])
        record_attrs = json.loads(row['RECORD_ATTRIBUTES'])
        
        span_id = trace_info.get('span_id')
        parent_span_id = record_info.get('parent_span_id')
        span_name = record_info.get('name', 'unknown')
        
        spans[span_id] = {
            'name': span_name,
            'parent_id': parent_span_id,
            'start_time': row['START_TIMESTAMP'],
            'record': record_info,
            'attributes': record_attrs,
            'full_content': record_attrs  # Store all attributes for concatenation
        }
    
    # Build hierarchy and concatenate content
    root_spans = [span_id for span_id, span in spans.items() 
                  if span['parent_id'] is None or span['parent_id'] not in spans]
    
    def print_hierarchy_with_content(span_id, level=0):
        if span_id not in spans:
            return ""
        
        span = spans[span_id]
        indent = "  " * level
        
        # Format all record_attributes content
        content_lines = []
        for key, value in span['attributes'].items():
            if value is not None and str(value).strip():  # Skip empty values
                content_lines.append(f"{key}: {value}")
        
        content_str = "\n".join([f"{indent}  {line}" for line in content_lines])
        
        span_header = f"{indent}{span['name']} ({span_id}) - {span['start_time']}"
        full_span_content = f"{span_header}\n{content_str}" if content_str else span_header
        
        print(full_span_content)
        
        # Find and process children
        children = [sid for sid, s in spans.items() if s['parent_id'] == span_id]
        for child_id in sorted(children, key=lambda x: spans[x]['start_time']):
            print_hierarchy_with_content(child_id, level + 1)
    
    print("\nSpan Hierarchy with Full Content:")
    print("=" * 80)
    for root_id in sorted(root_spans, key=lambda x: spans[x]['start_time']):
        print_hierarchy_with_content(root_id)
        print("-" * 80)
    
    return spans

def get_concatenated_trace_content(trace_df):
    """Get all record_attributes content concatenated in chronological order"""
    all_content = []
    
    for idx, row in trace_df.iterrows():
        record_attrs = json.loads(row['RECORD_ATTRIBUTES'])
        record_info = json.loads(row['RECORD'])
        
        span_name = record_info.get('name', 'unknown')
        timestamp = row['START_TIMESTAMP']
        
        content_block = f"\n[{timestamp}] {span_name}:\n"
        
        for key, value in record_attrs.items():
            if value is not None and str(value).strip():
                content_block += f"  {key}: {value}\n"
        
        all_content.append(content_block)
    
    return "\n".join(all_content)

# Usage:
trace_df = get_execution_trace(snowpark_session, request_id='57ec98b7-57de-4b05-abff-1560da5003c4')

# Hierarchical view with all content
span_hierarchy = build_span_hierarchy_with_content(trace_df)

# Linear concatenated view
assembled_trace = get_concatenated_trace_content(trace_df)
print("\nConcatenated Trace Content:")
print("=" * 80)
print(assembled_trace) # THIS IS THE TRACE TO BE USED IN THE EVALUATION PROMPT


Span Hierarchy with Full Content:
Agent (9445176b957fadc4) - 2025-10-01 22:09:58.752000
  ai.observability.record_id: 57ec98b7-57de-4b05-abff-1560da5003c4
  request_id: 57ec98b7-57de-4b05-abff-1560da5003c4
  snow.ai.observability.agent.thread_id: 2048699660006
  snow.ai.observability.database.id: 31272877
  snow.ai.observability.database.name: SNOWFLAKE_INTELLIGENCE
  snow.ai.observability.object.id: 31262585
  snow.ai.observability.object.name: SNOWFLAKE_DOCS_AND_KNOWLEDGE
  snow.ai.observability.object.type: Cortex Agent
  snow.ai.observability.object.version.id: 24
  snow.ai.observability.schema.id: 2048701987454
  snow.ai.observability.schema.name: AGENTS
  snow.ai.observability.span_kind: 1
  AgentV2RequestResponseInfo (b1799e48866df4e8) - 2025-10-01 22:09:58.752000
    ai.observability.record_id: 57ec98b7-57de-4b05-abff-1560da5003c4
    snow.ai.observability.agent.duration: 10857
    snow.ai.observability.agent.first_message_in_thread: can i create a clone of a table and add lcu